In [1]:
from parameters.dataset import BPI2012Parameters
from dataset import BPI2012Dataset
from utils.print import print_block
from utils.bpi2012 import remove_trail_steps, print_model_prediction_result, remove_tags_for_seq, get_example_data_with_removed_tail
import tensorflow as tf
from model import LSTMPredNextModel, LSTMScenarioCfModel
import numpy as np
from dice import EventLogDiCE
from parameters.training import TrainingParameters, LossParameters, OptimizerParameters
from utils.preprocessing import dataset_split
from cf_search import CfSearcher

In [2]:
### Load dataset
dataset = BPI2012Dataset(BPI2012Parameters()) 


| Preprocessed data loaded successfully: ./data/preprocessed/BPI_Challenge_2012_with_resource/AOW_CompleteOnly 


In [3]:
### Prepare the training dataset.
train_params = TrainingParameters(
    stop_epoch=20,
)
train_dataset, test_dataset, validation_dataset = dataset_split(
    list(range(len(dataset))),
    train_params.train_test_split_portion,
    seed=train_params.random_seed,
    shuffle=True
)
training_df = dataset.df.iloc[train_dataset]

In [4]:
example_vocab_activities, example_idx_activities, example_vocab_resources, example_idx_resources, example_amount = get_example_data_with_removed_tail(dataset, trace_idx=52, tail_length_to_remove=13)


====================Activity====================
| [['<SOS>', 'A_SUBMITTED_COMPLETE', 'A_PARTLYSUBMITTED_COMPLETE', 'A_PREACCEPTED_COMPLETE', 'W_Completeren aanvraag_COMPLETE', 'W_Completeren aanvraag_COMPLETE', 'A_ACCEPTED_COMPLETE', 'A_FINALIZED_COMPLETE', 'O_SELECTED_COMPLETE', 'O_CREATED_COMPLETE', 'O_SENT_COMPLETE', 'W_Completeren aanvraag_COMPLETE', 'O_SENT_BACK_COMPLETE', 'W_Nabellen offertes_COMPLETE', 'O_ACCEPTED_COMPLETE', 'A_APPROVED_COMPLETE', 'A_REGISTERED_COMPLETE', 'A_ACTIVATED_COMPLETE', 'W_Valideren aanvraag_COMPLETE']] 

====================Resource====================
| [['<SOS>', '112', '112', '112', '11180', '11201', '11201', '11201', '11201', '11201', '11201', '11201', '11049', '11049', '10629', '10629', '10629', '10629', '10629']] 

====================Amount====================
| [15500.0] 

====================Activity (Tail removed)====================
| ['<SOS>', 'A_SUBMITTED_COMPLETE', 'A_PARTLYSUBMITTED_COMPLETE', 'A_PREACCEPTED_COMPLETE', 'W_Completeren a

In [5]:
##### load prediction model #####
pred_model = LSTMPredNextModel.load(
    "./SavedModels/%s" % (
    "0.8175_LSTMPredNextModel_AOW_CompleteOnly_2021-07-01 20:45:16.353467" 
    )
)
_ = pred_model(**pred_model.get_example_input())
pred_model.summary()


| Model parameters loaded successfully from: ./SavedModels/0.8175_LSTMPredNextModel_AOW_CompleteOnly_2021-07-01 20:45:16.353467  

| Vocab loaded successfully from: ./SavedModels/0.8175_LSTMPredNextModel_AOW_CompleteOnly_2021-07-01 20:45:16.353467  

| Model loaded successfully from: ./SavedModels/0.8175_LSTMPredNextModel_AOW_CompleteOnly_2021-07-01 20:45:16.353467  
Model: "LSTMPredNextModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  832       
_________________________________________________________________
embedding_1 (Embedding)      multiple                  9216      
_________________________________________________________________
lstm (LSTM)                  multiple                  24832     
_________________________________________________________________
lstm_1 (LSTM)                multiple                  33024     
________

In [6]:
##### Get model output #####
example_activities_input = tf.constant(example_idx_activities)
example_resources_input = tf.constant(example_idx_resources)
example_amount_input = tf.constant(example_amount)

predicted_df = print_model_prediction_result(pred_model, example_activities_input, example_resources_input, example_amount_input)


========================================Predict result========================================
| Predicted activity with highest probability (0.69) is "W_Completeren aanvraag_COMPLETE" 



<PAD>                                      1.174409e-07
<EOS>                                      3.132308e-05
<SOS>                                      1.881654e-07
A_ACCEPTED_COMPLETE                        2.363838e-01
A_ACTIVATED_COMPLETE                       1.805911e-06
A_APPROVED_COMPLETE                        2.348171e-06
A_CANCELLED_COMPLETE                       6.621396e-02
A_DECLINED_COMPLETE                        1.187625e-02
A_FINALIZED_COMPLETE                       9.158659e-06
A_PARTLYSUBMITTED_COMPLETE                 1.821592e-07
A_PREACCEPTED_COMPLETE                     1.974105e-07
A_REGISTERED_COMPLETE                      2.050257e-06
A_SUBMITTED_COMPLETE                       1.642594e-06
O_ACCEPTED_COMPLETE                        1.552491e-05
O_CANCELLED_COMPLETE      

In [7]:
## initialise searcher
searcher = CfSearcher(training_df, pred_model)

In [8]:
### find cf
cf_df = searcher.search(example_vocab_activities, desired="A_ACCEPTED_COMPLETE", amount=example_amount[0], replace_amount=5000)

In [9]:
cf_df

,activity,activity_vocab,caseid,amount,resource,resource_vocab,predicted_vocab,predicted_value
52,"[2, 12, 9, 10, 22, 22]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",173844,5000,"[70, 53, 53, 53, 50, 55]","[<SOS>, 112, 112, 112, 11180, 11201]",W_Completeren aanvraag_COMPLETE,0.709127
61,"[2, 12, 9, 10, 22, 22, 22, 22, 22]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",173871,5000,"[70, 53, 53, 53, 71, 55, 18, 18, 20]","[<SOS>, 112, 112, 112, UNKNOWN, 11201, 10889, ...",W_Completeren aanvraag_COMPLETE,0.913418
6714,"[2, 12, 9, 10, 20]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",194583,5000,"[70, 53, 53, 30, 30]","[<SOS>, 112, 112, 10939, 10939]",W_Completeren aanvraag_COMPLETE,0.532845
1031,"[2, 12, 9, 10, 20]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",176927,5000,"[70, 53, 53, 38, 38]","[<SOS>, 112, 112, 11009, 11009]",W_Completeren aanvraag_COMPLETE,0.690654
7405,"[2, 12, 9, 10]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",196777,5000,"[70, 53, 53, 53]","[<SOS>, 112, 112, 112]",W_Completeren aanvraag_COMPLETE,0.571807
583,"[2, 12, 9, 10, 22]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",175528,5000,"[70, 53, 53, 53, 22]","[<SOS>, 112, 112, 112, 10912]",W_Completeren aanvraag_COMPLETE,0.653006
7554,"[2, 12, 9, 10, 20]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",197234,5000,"[70, 53, 53, 25, 25]","[<SOS>, 112, 112, 10929, 10929]",W_Completeren aanvraag_COMPLETE,0.633907
9470,"[2, 12, 9, 10, 20]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",203206,5000,"[70, 53, 53, 21, 21]","[<SOS>, 112, 112, 10910, 10910]",W_Completeren aanvraag_COMPLETE,0.661027
7786,"[2, 12, 9, 10]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",197948,5000,"[70, 53, 53, 53]","[<SOS>, 112, 112, 112]",W_Completeren aanvraag_COMPLETE,0.571807
